In [25]:
def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """

    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    # Find all instances of naked twins
    for unit in unitlist:
        naked_twins = []
        possible_twins = [box for box in unit if len(values[box])==2]
        for idx, a in enumerate(possible_twins):
            for b in possible_twins[idx+1:]:
                if values[a]==values[b]:
                    naked_twins.append(a)
                    naked_twins.append(b)                    
        
        print(naked_twins)
    # Eliminate the naked twins as possibilities for their peers        
        for box in list(set(naked_twins)):
            for pbox in list(set(unit)-set(naked_twins)):
                for digit in values[box]:            
                    values[pbox] = values[pbox].replace(digit,'') 
    
    return values  


def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s+t for s in A for t in B]

def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))

def display(values):
    """
    Display the values as a 2-D grid.
    Args:
        values(dict): The sudoku in dictionary form
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def eliminate(values):
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

def only_choice(values):
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

def reduce_puzzle(values):
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

def search(values):
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    
    if values is False:
        return False
    if all(len(values[s])==1 for s in boxes):
        return values
        
    # Choose one of the unfilled squares with the fewest possibilities
    min_len = min([len(values[s]) for s in boxes if len(values[s])>1])
    
    min_index = [k for k, v in values.items() if len(v)==min_len][0]
    # Now use recursion to solve each one of the resulting sudokus, and if one returns a value (not False), return that answer!
    # for k in fewest_units:
    for v in values[min_index]:
        new_sudoku = values.copy()
        new_sudoku[min_index] = v
        attempt = search(new_sudoku)
        if attempt:
            return attempt

def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    return search(grid_values(grid))

    
assignments = []
    
rows = 'ABCDEFGHI'
cols = '123456789'
boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
diag_units = [[x+y for (x, y) in zip(rows, cols)], [x+y for (x, y) in zip(rows, cols[::-1])]]
diag_units=[]


unitlist = row_units + column_units + square_units + diag_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

# if __name__ == '__main__':
diag_sudoku_grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
diag_sudoku_grid = '.......5632....7.15......2..15..........92....8.7.............445........3.......'
# res = solve(diag_sudoku_grid)
if res:
    display(res)

try:
    from visualize import visualize_assignments
    visualize_assignments(assignments)

except SystemExit:
    pass
except:
    print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.


In [28]:
values = {"E7": "1234568", "H6": "23568", "E3": "25689", "F5": "1345689", "I6":
"4", "F7": "1234568", "D2": "4", "G6": "9", "A4": "5", "B9": "9",
"A1": "2468", "I3": "25678", "C8": "13456", "E1": "123689", "H5":
"35678", "B8": "8", "I4": "23678", "C6": "368", "H2": "23568", "D6":
"23568", "I1": "123678", "A7": "2346", "G8": "1234567", "B6": "1",
"H9": "2345678", "D1": "123689", "G7": "1234568", "E4": "234678",
"B4": "46", "F3": "25689", "H3": "2456789", "B1": "5", "E2": "123568",
"E8": "12345679", "G3": "245678", "G9": "12345678", "C4": "9", "F8":
"1234569", "A9": "2346", "D7": "123568", "I9": "1235678", "I5":
"35678", "G1": "1234678", "I8": "123567", "A8": "2346", "D5":
"1356789", "C3": "4678", "C2": "68", "A5": "3468", "G4": "23678",
"H1": "2346789", "E5": "13456789", "D8": "1235679", "B7": "7", "A2":
"9", "C5": "2", "G2": "123568", "C7": "13456", "F6": "23568", "F1":
"123689", "G5": "35678", "B2": "2", "F9": "1234568", "A3": "1", "B3":
"3", "F2": "7", "H8": "234567", "E6": "23568", "C1": "4678", "I2":
"123568", "E9": "12345678", "A6": "7", "B5": "46", "H7": "234568",
"D3": "25689", "D9": "1235678", "D4": "23678", "F4": "23468", "C9":
"13456", "H4": "1", "I7": "9"}

naked_twins(values)

[]
['B4', 'B5']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['B4', 'B5']
[]
[]
[]
[]
[]
[]
[]


{'A1': '2468',
 'A2': '9',
 'A3': '1',
 'A4': '5',
 'A5': '38',
 'A6': '7',
 'A7': '2346',
 'A8': '2346',
 'A9': '2346',
 'B1': '5',
 'B2': '2',
 'B3': '3',
 'B4': '46',
 'B5': '46',
 'B6': '1',
 'B7': '7',
 'B8': '8',
 'B9': '9',
 'C1': '4678',
 'C2': '68',
 'C3': '4678',
 'C4': '9',
 'C5': '2',
 'C6': '38',
 'C7': '13456',
 'C8': '13456',
 'C9': '13456',
 'D1': '123689',
 'D2': '4',
 'D3': '25689',
 'D4': '23678',
 'D5': '1356789',
 'D6': '23568',
 'D7': '123568',
 'D8': '1235679',
 'D9': '1235678',
 'E1': '123689',
 'E2': '123568',
 'E3': '25689',
 'E4': '234678',
 'E5': '13456789',
 'E6': '23568',
 'E7': '1234568',
 'E8': '12345679',
 'E9': '12345678',
 'F1': '123689',
 'F2': '7',
 'F3': '25689',
 'F4': '23468',
 'F5': '1345689',
 'F6': '23568',
 'F7': '1234568',
 'F8': '1234569',
 'F9': '1234568',
 'G1': '1234678',
 'G2': '123568',
 'G3': '245678',
 'G4': '23678',
 'G5': '35678',
 'G6': '9',
 'G7': '1234568',
 'G8': '1234567',
 'G9': '12345678',
 'H1': '2346789',
 'H2': '23568',
 

In [18]:
res

False

In [8]:
diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
res = search(grid_values(diag_sudoku_grid))

display(res)

2 6 4 |8 3 7 |1 5 9 
7 9 3 |1 5 6 |2 4 8 
5 8 1 |9 2 4 |3 7 6 
------+------+------
4 5 6 |3 7 8 |9 2 1 
3 1 2 |4 9 5 |6 8 7 
9 7 8 |6 1 2 |4 3 5 
------+------+------
1 4 7 |5 6 3 |8 9 2 
6 3 5 |2 8 9 |7 1 4 
8 2 9 |7 4 1 |5 6 3 
